In [ ]:
#Let's analyze the Tourism industry in Tanzania
import pandas as pd

In [ ]:
#First we review the CSV
survey = pd.read_csv("./2014TourismSurvey.csv")

In [ ]:
survey.columns #what were features of this survey?
#Don't understand what PAC means... at first glance it means if something is included in a package...
#the survey is now part of the folder for extra clarity

In [ ]:
survey[['AGE_GROUP', 'IF_TRAVEL_ALONE', 'TRAVELLING_WITH', 'AGE_GROUP1_SIZE (<18)', 'AGE_GROUP2_SIZE (18-24)', 'AGE_GROUP3_SIZE (25-44)',
       'AGE_GROUP4_SIZE (45-64)', 'AGE_GROUP5_SIZE (65+)']]

#Trying to better understand age group and seems like the only reliable data point is AGE_GROUP and maybe
#IF_TRAVEL_ALONE and TRAVELLING_WITH.

In [ ]:
#Check values that were recorded for features using unique

survey['AGE_GROUP'].unique() #Originally weird thing were all the 8 in the ranges got changed to
#'Others (please specify)'. Find and replace gone wrong I imagine...
#Yeah the person who found and replaced 8s randomly is next. level.

In [ ]:
#Okay - let's clean up some of the data thus far. We can't really trust the individual group size columns
#since interpreting what it means appears to be a messy process at best, so I'm opting to drop them for now
#Also found some select other columns to be somewhat nonsense

survey.drop(['AGE_GROUP1_SIZE (<18)', 'AGE_GROUP2_SIZE (18-24)', 'AGE_GROUP3_SIZE (25-44)',
       'AGE_GROUP4_SIZE (45-64)', 'AGE_GROUP5_SIZE (65+)', 'NIGHTS_SPENT', 'PAC_OTHER_ITEM',
            'ACCOMODATION', 'TZ_COST_COMPARISON', 'OTHER1', 'OTHER2', 'OTHER3', 'VERIFICATION_FLAG',
             'TRAVEL_ELSE_AFRICA'], axis=1, inplace=True)

In [ ]:
#We also need to replace broken AGE_GROUP values
survey.replace({'AGE_GROUP': {'<1Others (please specify)': '<18', '1Others (please specify)-24': '18-24'}},
              inplace=True)

In [ ]:
#Great! Now let's see what other things we might want to clean up. Made functions of my most common clean up tasks
#on this dataset.

#change the column name for Tour Arrangement
survey.rename(columns={'TOUR_ARRANGEMENT': 'tour_package'}, inplace=True)

def unique(column, df):
    return df[column].unique()

def return_eight(column, df):
    #pythonic capture of all the crazy values
    x = [s for s in unique(column, df) if 'O' in str(s)]

    y = [int(s.replace('Others (please specify)', '8')) for s in x]

    rep_dict = dict(zip(x,y))

    #change the values
    df.replace({column: rep_dict}, inplace=True)

def take_commas(column, df):
    x = [s for s in unique(column, df) if ',' in str(s)]

    y = [float(s.replace(',', '')) for s in x]

    rep_dict = dict(zip(x,y))

    #change the values
    df.replace({column: rep_dict}, inplace=True)

def make_numeric(column, df):
    x = [s for s in unique(column, df) if type(s) == str]

    y = [float(s) for s in x]

    rep_dict = dict(zip(x,y))

    #change the values
    df.replace({column: rep_dict}, inplace=True)

In [ ]:
#Data is still dirty and here is yet another example of an inconsistency...travelling alone but with someone?
#traveling but 0 total travellers? You'll have to trust some feature over others it'd appear.

survey['TOTAL_TRAVELLERS'] = survey['NO_FEMALE'] + survey['NO_MALE']
survey[['IF_TRAVEL_ALONE', 'TRAVELLING_WITH', 'TOTAL_TRAVELLERS']][(survey.loc[:,'IF_TRAVEL_ALONE']==1) & (survey.loc[:,'TOTAL_TRAVELLERS']!=1)]

In [ ]:
#A basic analysis: Who spends more per visit on average? First comers, or returners?
howSpent = survey[['IF_FIRST_TRIP_TZ', 'CURRENCY2', 'SPENT_AMOUNT_TZ']]

In [ ]:
currencies = {'TZS': lambda x: x*0.00044, 'USD': lambda x: x, 'AUD': lambda x: x*.77, 'EUR':lambda x: x*1.23,'ZAR': lambda x: x*.083,
                  'GBP': lambda x: x*1.4, 'NOK': lambda x: x*.13, 'KES': lambda x: x*.01, 'AED': lambda x: x*.27,
                  'QAR': lambda x: x*.27, 'CAD': lambda x: x*.78, 'DKK': lambda x: x*.16, 'CHF': lambda x: x*1.03,
              'SAR': lambda x: x*.27, 'UGX': lambda x: x*0.00027, 'INR': lambda x: x*.015, 'SEK': lambda x: x*.12,
              'ZMK': lambda x: x*.11, 'NZD': lambda x: x*.72, 'JYP': lambda x: x*.0093, 'ROL': lambda x: x*.26}

for k, v in currencies.items():
    howSpent.loc[howSpent["CURRENCY2"]==k, "spentInUSD"] = howSpent['SPENT_AMOUNT_TZ'].apply(v)

In [ ]:
howSpent.groupby('IF_FIRST_TRIP_TZ').mean()